In [1]:
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
from selenium import webdriver
import time
import pandas as pd
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


In [20]:
DRIVER_PATH = "/home/zaki/Downloads/chromedriver_linux64/chromedriver"
driver = webdriver.Chrome(executable_path=DRIVER_PATH)
driver.get('https://www.techwithtim.net/')

#time.sleep(5)
#

In [81]:
def get_jobs(keyword, num_jobs, verbose=True):
    
    '''Gathers jobs as a dataframe, scraped from Glassdoor'''
    
    #Initializing the webdriver
    options = webdriver.ChromeOptions()
    
    #Uncomment the line below if you'd like to scrape without a new Chrome window every time.
    #options.add_argument('headless')
    
    #Change the path to where chromedriver is in your home folder.
    path = "./chromedriver"
    driver = webdriver.Chrome(executable_path=path, options=options)
    driver.set_window_size(1920, 1080) #(1120, 1000)
    
    #driver.execute_script("document.body.style.zoom='25%'")
    url = 'https://www.glassdoor.com/Job/jobs.htm?sc.keyword=' + keyword + '&locT=C&locId=1147401&locKeyword=San%20Francisco,%20CA&jobType=all&fromAge=-1&minSalary=0&includeNoSalaryJobs=false&radius=100&cityId=-1&minRating=0.0&industryId=-1&sgocId=-1&seniorityType=all&companyId=-1&employerSizes=0&applicationType=0&remoteWorkType=0'
    driver.get(url)
    #driver.execute_script("document.body.style.zoom='0.5'")
    jobs = []

    while len(jobs) < num_jobs:  #If true, should be still looking for new jobs.

        #Let the page load. Change this number based on your internet speed.
        #Or, wait until the webpage is loaded, instead of hardcoding it.
        time.sleep(1)

        #Test for the "Sign Up" prompt and get rid of it.
       
        try:
            driver.find_element_by_class_name("selected").click()
        except   NoSuchElementException:
            pass

        time.sleep(4)

        try:
            driver.find_element_by_css_selector('[alt="Close"]').click()  #clicking to the X.
            print('X out worked')
        except NoSuchElementException:
            print('X out failed')
            pass

        
        #Going through each job in this page
        job_buttons = driver.find_elements_by_css_selector("li.react-job-listing")  #jl for Job Listing. These are the buttons we're going to click.
        for job_button in job_buttons:  

            print("Progress: {}".format("" + str(len(jobs)) + "/" + str(num_jobs)))
            if len(jobs) >= num_jobs:
                break
                
            job_button.click()  #You might
            time.sleep(3)
                
                #___________ code to kill the sign-up pop-up after it render on screen
                # if not found_popup:
            try:
                    driver.find_element_by_css_selector('[alt="Close"]').click()
                    # print("&&& line 89")
                    # found_popup = True
            except NoSuchElementException:
                    # print("&&& line 92")
                    pass
            '''
            try:   
                job_button.click()  #You might
            except:
                
                #___________ code to kill the sign-up pop-up after it render on screen
                # if not found_popup:
                try:
                    driver.find_element_by_css_selector('[alt="Close"]').click()
                    # print("&&& line 89")
                    # found_popup = True
                except NoSuchElementException:
                    # print("&&& line 92")
                    pass
            '''
            #time.sleep(1)
            time.sleep(.1)

            
            #pass
            collected_successfully = False
            
            while not collected_successfully:
                try:
                    company_name = driver.find_element_by_xpath('.//div[@class="css-87uc0g e1tk4kwz1"]').text  
                    location = driver.find_element_by_xpath('.//div[@class="css-56kyx5 e1tk4kwz5"]').text
                    job_title = driver.find_element_by_xpath('.//div[@class="css-1vg6q84 e1tk4kwz4"]').text
                    job_description = driver.find_element_by_xpath('.//div[@class="jobDescriptionContent desc"]').text
                    collected_successfully = True
                except:
                    time.sleep(5)

            try:
                salary_estimate = driver.find_element_by_xpath('.//div[@class="css-1bluz6i e2u4hf13"]').text
            except NoSuchElementException:
                salary_estimate = -1 #You need to set a "not found value. It's important."
            
            try:
                rating = driver.find_element_by_xpath('.//span[@class="css-1m5m32b e1tk4kwz2"]').text
            except NoSuchElementException:
                rating = -1 #You need to set a "not found value. It's important."

            #Printing for debugging
            if verbose:
                print("Job Title: {}".format(job_title))
                print("Salary Estimate: {}".format(salary_estimate))
                print("Job Description: {}".format(job_description[:500]))
                print("Rating: {}".format(rating))
                print("Company Name: {}".format(company_name))
                print("Location: {}".format(location))

            #Going to the Company tab...
            #clicking on this:
            #<div class="tab" data-tab-type="overview"><span>Company</span></div>
            try:
                company = driver.find_elements_by_xpath('.//div[@class="d-flex justify-content-start css-rmzuhb e1pvx6aw0"]//span[@class="css-i9gxme e1pvx6aw2"]')
                lable = driver.find_elements_by_xpath('.//div[@class="d-flex justify-content-start css-rmzuhb e1pvx6aw0"]//span[@class="css-1taruhi e1pvx6aw1"]') #.text
                lable = [str(c.text) for c in lable]
                tap = {key: value.text for key, value in zip(lable, company)}
            
                size = tap['Size'] if 'Size' in tap else -1  
                founded = tap['Founded'] if 'Founded' in tap else -1  
                type_of_ownership = tap['Type'] if 'Type' in tap else -1  
                industry = tap['Industry'] if 'Industry' in tap else -1  
                sector = tap['Sector'] if 'Sector' in tap else -1  
                revenue = tap['Revenue'] if 'Revenue' in tap else -1  
                headquarters =  -1 

            except NoSuchElementException:  #Rarely, some job postings do not have the "Company" tab.
                headquarters = -1
                size = -1
                founded = -1
                type_of_ownership = -1
                industry = -1
                sector = -1
                revenue = -1
                competitors = -1

                
            if verbose:
                print("Headquarters: {}".format(headquarters))
                print("Size: {}".format(size))
                print("Founded: {}".format(founded))
                print("Type of Ownership: {}".format(type_of_ownership))
                print("Industry: {}".format(industry))
                print("Sector: {}".format(sector))
                print("Revenue: {}".format(revenue))
                # print("Competitors: {}".format(competitors))
                print("@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@")

            jobs.append({"Job Title" : job_title,
            "Salary Estimate" : salary_estimate,
            "Job Description" : job_description,
            "Rating" : rating,
            "Company Name" : company_name,
            "Location" : location,
            "Headquarters" : headquarters,
            "Size" : size,
            "Founded" : founded,
            "Type of ownership" : type_of_ownership,
            "Industry" : industry,
            "Sector" : sector,
            "Revenue" : revenue})
            #"Competitors" : competitors
            #add job to jobs

        #Clicking on the "next page" button
        try:
            driver.find_element_by_xpath('.//li[@class="css-114lpwu e1gri00l4"]//a').click()
        except ElementClickInterceptedException:
            print("Scraping terminated before reaching target number of jobs. Needed {}, got {}.".format(num_jobs, len(jobs)))
            break

    return pd.DataFrame(jobs)  #This line converts the dictionary object into a pandas DataFrame.

In [82]:
#This line will open a new chrome window and start the scraping.
df = get_jobs('data scientist',10)
df

X out failed
Progress: 0/10
Job Title: Data Scientist
Salary Estimate: $94,444 /yr (est.)
Job Description: At eXcell, we set you up for job success right from the start. Our precision recruiting process aligns the right fit for the right people.

Description:
We are looking for a Data Scientist who will support our factory and delivery leadership teams with insights gained from analyzing company data. The ideal candidate is adept at using large data sets to find opportunities for service and process optimization and using models to test the effectiveness of different courses of action. They must have
Rating: 3.9
Company Name: eXcell
3.9
Location: Sacramento, CA
Headquarters: -1
Size: 201 to 500 Employees
Founded: 2001
Type of Ownership: Subsidiary or Business Segment
Industry: IT Services
Sector: Information Technology
Revenue: $25 to $50 million (USD)
@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
Progress: 1/10
Job Title: Director, Data Scientist
Salary Estimate: $142,420 /yr (

ElementClickInterceptedException: Message: element click intercepted: Element <li class="react-job-listing css-wp148e eigr9kq3" data-brandviews="BRAND:n=jsearch-job-listing:eid=2452794:jlid=4120147026" data-id="4120147026" data-adv-type="EMPLOYER" data-is-organic-job="false" data-ad-order-id="1044077" data-sgoc-id="1019" data-is-easy-apply="true" data-normalize-job-title="Senior Research Scientist" data-job-loc="San Jose, CA" data-job-loc-id="1147436" data-job-loc-type="C" data-selected="false" data-test="jobListing" style="border-bottom:0" data-triggered-brandview="">...</li> is not clickable at point (531, 863). Other element would receive the click: <div id="onetrust-group-container" class="ot-sdk-eight ot-sdk-columns">...</div>
  (Session info: chrome=91.0.4472.101)


In [78]:
df

,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,Industry,Sector,Revenue
0,Data Scientist - Intermediate,-1,The Yield Modeling team at The Climate Corpora...,4.1,Bayer\n4.1,"San Francisco, CA",-1,10000+ Employees,1863,Company - Private,Biotech & Pharmaceuticals,Biotech & Pharmaceuticals,$10+ billion (USD)
1,Data Scientist - Intermediate,-1,The Yield Modeling team at The Climate Corpora...,4.1,Bayer\n4.1,"San Francisco, CA",-1,10000+ Employees,1863,Company - Private,Biotech & Pharmaceuticals,Biotech & Pharmaceuticals,$10+ billion (USD)
2,Data Scientist - Intermediate,-1,The Yield Modeling team at The Climate Corpora...,4.1,Bayer\n4.1,"San Francisco, CA",-1,10000+ Employees,1863,Company - Private,Biotech & Pharmaceuticals,Biotech & Pharmaceuticals,$10+ billion (USD)
3,Data Scientist - Intermediate,-1,The Yield Modeling team at The Climate Corpora...,4.1,Bayer\n4.1,"San Francisco, CA",-1,10000+ Employees,1863,Company - Private,Biotech & Pharmaceuticals,Biotech & Pharmaceuticals,$10+ billion (USD)
4,Data Scientist - Intermediate,-1,The Yield Modeling team at The Climate Corpora...,4.1,Bayer\n4.1,"San Francisco, CA",-1,10000+ Employees,1863,Company - Private,Biotech & Pharmaceuticals,Biotech & Pharmaceuticals,$10+ billion (USD)
5,Data Scientist - Intermediate,-1,The Yield Modeling team at The Climate Corpora...,4.1,Bayer\n4.1,"San Francisco, CA",-1,10000+ Employees,1863,Company - Private,Biotech & Pharmaceuticals,Biotech & Pharmaceuticals,$10+ billion (USD)
6,Data Scientist - Intermediate,-1,The Yield Modeling team at The Climate Corpora...,4.1,Bayer\n4.1,"San Francisco, CA",-1,10000+ Employees,1863,Company - Private,Biotech & Pharmaceuticals,Biotech & Pharmaceuticals,$10+ billion (USD)
7,Data Scientist - Intermediate,-1,The Yield Modeling team at The Climate Corpora...,4.1,Bayer\n4.1,"San Francisco, CA",-1,10000+ Employees,1863,Company - Private,Biotech & Pharmaceuticals,Biotech & Pharmaceuticals,$10+ billion (USD)
8,Data Scientist - Intermediate,-1,The Yield Modeling team at The Climate Corpora...,4.1,Bayer\n4.1,"San Francisco, CA",-1,10000+ Employees,1863,Company - Private,Biotech & Pharmaceuticals,Biotech & Pharmaceuticals,$10+ billion (USD)
9,Data Scientist - Intermediate,-1,The Yield Modeling team at The Climate Corpora...,4.1,Bayer\n4.1,"San Francisco, CA",-1,10000+ Employees,1863,Company - Private,Biotech & Pharmaceuticals,Biotech & Pharmaceuticals,$10+ billion (USD)
